<a href="https://colab.research.google.com/github/safaet/mufti-llm/blob/main/MiniLLMWithDatabaseMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================
# 🧠 Mini LLM from scratch (step-by-step, no class)
# =====================================================

import sqlite3, io, numpy as np, torch, torch.nn.functional as F, os, random

# -------------------------------
# 0️⃣ Setup
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# -------------------------------
# 1️⃣ Model configuration
# -------------------------------
vocab_size = 2048
d_model = 32
n_heads = 4
ff_hidden = 64
max_seq_len = 24
pad_token_id, bos_token_id, eos_token_id = 0, 1, 2
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# -------------------------------
# 2️⃣ Tokenization (hash-based)
# -------------------------------
def tokenize(text):
    tokens = text.lower().split()
    ids = []
    for t in tokens[:max_seq_len]:
        h = abs(hash(t)) % (vocab_size - 3) + 3
        ids.append(h)
    return ids

# -------------------------------
# 3️⃣ Embedding initialization
# -------------------------------
embedding = torch.nn.Embedding(vocab_size, d_model).to(device)
pos_embedding = torch.nn.Embedding(max_seq_len, d_model).to(device)

# -------------------------------
# 4️⃣ Transformer weights (manual)
# -------------------------------
W_q = torch.randn(d_model, d_model, device=device) / (d_model ** 0.5)
W_k = torch.randn(d_model, d_model, device=device) / (d_model ** 0.5)
W_v = torch.randn(d_model, d_model, device=device) / (d_model ** 0.5)
W_o = torch.randn(d_model, d_model, device=device) / (d_model ** 0.5)
W_ff1 = torch.randn(d_model, ff_hidden, device=device) * (2.0 / (d_model ** 0.5))
b_ff1 = torch.zeros(ff_hidden, device=device)
W_ff2 = torch.randn(ff_hidden, d_model, device=device) * (2.0 / (ff_hidden ** 0.5))
b_ff2 = torch.zeros(d_model, device=device)
ln1 = torch.nn.LayerNorm(d_model).to(device)
ln2 = torch.nn.LayerNorm(d_model).to(device)
W_out = torch.randn(d_model, vocab_size, device=device) / (d_model ** 0.5)
b_out = torch.zeros(vocab_size, device=device)

for t in [embedding.weight, pos_embedding.weight, W_q, W_k, W_v, W_o,
          W_ff1, b_ff1, W_ff2, b_ff2, W_out, b_out]:
    if isinstance(t, torch.Tensor):
        t.requires_grad = False

# -------------------------------
# 5️⃣ Transformer block
# -------------------------------
def single_transformer_block(x):
    batch, seq_len, _ = x.shape
    Q = x @ W_q
    K = x @ W_k
    V = x @ W_v
    head_dim = d_model // n_heads

    def split_heads(t):
        t = t.view(batch, seq_len, n_heads, head_dim)
        return t.permute(0, 2, 1, 3)

    Qh = split_heads(Q)
    Kh = split_heads(K)
    Vh = split_heads(V)

    attn_scores = torch.matmul(Qh, Kh.transpose(-2, -1)) / (head_dim ** 0.5)
    mask = torch.tril(torch.ones((seq_len, seq_len), device=device)).unsqueeze(0).unsqueeze(0)
    attn_scores = attn_scores.masked_fill(mask == 0, float('-1e9'))
    attn_weights = F.softmax(attn_scores, dim=-1)
    attn_out = torch.matmul(attn_weights, Vh)
    attn_out = attn_out.permute(0, 2, 1, 3).contiguous().view(batch, seq_len, d_model)
    attn_out = attn_out @ W_o

    x2 = ln1(x + attn_out)
    ff = F.relu(x2 @ W_ff1 + b_ff1) @ W_ff2 + b_ff2
    x3 = ln2(x2 + ff)
    return x3

# -------------------------------
# 6️⃣ Forward pass for logits
# -------------------------------
def forward_logits(token_ids):
    seq_len = len(token_ids)
    toks = torch.tensor(token_ids, dtype=torch.long, device=device).unsqueeze(0)
    pos_ids = torch.tensor(list(range(seq_len)), dtype=torch.long, device=device).unsqueeze(0)
    x = embedding(toks) + pos_embedding(pos_ids)
    x = single_transformer_block(x)
    logits = x @ W_out + b_out
    return logits

# -------------------------------
# 7️⃣ Text generation
# -------------------------------
def generate(prompt_text, max_new_tokens=8, temperature=1.0, top_k=30):
    token_ids = tokenize(prompt_text)
    token_ids = token_ids[-(max_seq_len-1):]
    for _ in range(max_new_tokens):
        logits = forward_logits(token_ids)
        last_logits = logits[0, -1, :] / (temperature if temperature>0 else 1.0)
        K = min(top_k, last_logits.shape[0])
        values, indices = torch.topk(last_logits, k=K)
        probs = F.softmax(values, dim=-1)
        next_id = indices[torch.multinomial(probs, num_samples=1).item()].item()
        token_ids.append(next_id)
        if next_id == eos_token_id:
            break
        if len(token_ids) > max_seq_len:
            token_ids = token_ids[-max_seq_len:]
    return token_ids

# =====================================================
# 💾 Simple Database Memory (SQLite)
# =====================================================
db_path = "mini_llm_store.sqlite"
if os.path.exists(db_path):
    os.remove(db_path)
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("CREATE TABLE kv (id INTEGER PRIMARY KEY AUTOINCREMENT, text TEXT, emb BLOB)")
conn.commit()

# -------------------------------
# Helper: vector save/load
# -------------------------------
def numpy_to_blob(x):
    memfile = io.BytesIO()
    np.save(memfile, x)
    memfile.seek(0)
    return memfile.read()

def blob_to_numpy(b):
    memfile = io.BytesIO(b)
    memfile.seek(0)
    return np.load(memfile)

# -------------------------------
# Compute sentence embedding
# -------------------------------
def sentence_embedding(text):
    ids = tokenize(text)
    if len(ids) == 0:
        return np.zeros(d_model, dtype=np.float32)
    toks = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)
    pos_ids = torch.tensor(list(range(len(ids))), dtype=torch.long, device=device).unsqueeze(0)
    with torch.no_grad():
        x = embedding(toks) + pos_embedding(pos_ids)
        x = single_transformer_block(x)
        vec = x.mean(dim=1).cpu().numpy().astype(np.float32)
    return np.squeeze(vec)

def store_sentence(text):
    emb = sentence_embedding(text)
    b = numpy_to_blob(emb)
    c.execute("INSERT INTO kv (text, emb) VALUES (?, ?)", (text, b))
    conn.commit()
    return c.lastrowid

def retrieve_similar(query, top_k=3):
    q_emb = sentence_embedding(query)
    c.execute("SELECT id, text, emb FROM kv")
    rows = c.fetchall()
    results = []
    for rid, txt, b in rows:
        emb = blob_to_numpy(b)
        denom = (np.linalg.norm(q_emb) * np.linalg.norm(emb))
        sim = float(np.dot(q_emb, emb) / denom) if denom > 0 else 0.0
        results.append((rid, txt, sim))
    results.sort(key=lambda x: x[2], reverse=True)
    return results[:top_k]

# =====================================================
# 🧩 Demo: store, retrieve, generate
# =====================================================
sample_sentences = [
    "The human brain works through electrical and chemical signals.",
    "Neurons communicate by sending electrical impulses called action potentials.",
    "Synapses are connections between neurons where neurotransmitters carry signals.",
    "The brain consumes a lot of energy and oxygen to function properly.",
    "Learning changes the strength of synapses in a process called synaptic plasticity."
]
for s in sample_sentences:
    store_sentence(s)

demo_query = "how do neurons communicate"
retrieved = retrieve_similar(demo_query, top_k=2)
print("\nRetrieved top context:")
for rid, txt, sim in retrieved:
    print(f" id={rid} sim={sim:.4f} text={txt}")

context = " ".join([txt for (_, txt, _) in retrieved])
prompt = context + " " + demo_query

print("\nPrompt to generator:\n", prompt)
generated_ids = generate(prompt, max_new_tokens=6, temperature=0.8, top_k=20)
print("\nGenerated token ids (last 12):", generated_ids[-12:])
store_sentence("AUTO: " + " ".join(map(str, generated_ids[-12:])))
print("\nStored generated response into DB.")
print("\n✅ Done. You can now modify the bottom part for interactive chat!")


Device: cpu

Retrieved top context:
 id=3 sim=0.8934 text=Synapses are connections between neurons where neurotransmitters carry signals.
 id=1 sim=0.8921 text=The human brain works through electrical and chemical signals.

Prompt to generator:
 Synapses are connections between neurons where neurotransmitters carry signals. The human brain works through electrical and chemical signals. how do neurons communicate

Generated token ids (last 12): [1697, 789, 1689, 74, 1880, 1139, 1052, 804, 192, 1112, 924, 1658]

Stored generated response into DB.

✅ Done. You can now modify the bottom part for interactive chat!


In [ ]:
while True:
    user_query = input("You: ")
    if user_query.lower() in ["exit", "quit"]:
        break
    retrieved = retrieve_similar(user_query)
    # Modify the line below to unpack only the text
    context = " ".join([txt for txt in retrieved])
    prompt = context + " " + user_query
    generated = generate(prompt, max_new_tokens=8)
    print("LLM:", generated[-8:])

You: what is your name
LLM: ame? the
You: safaet
LLM: ure. the
You: exit


In [ ]:
# =============================
# MINI LLM WITH READABLE OUTPUT
# =============================

import torch
import torch.nn as nn
import torch.nn.functional as F
import sqlite3
import numpy as np

# ---------- DEVICE ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ---------- PARAMETERS ----------
vocab = {}
reverse_vocab = {}
vocab_size = 1  # will grow dynamically
max_seq_len = 50
d_model = 64
num_heads = 2
d_ff = 128

# ---------- TOKENIZER ----------
def tokenize(text):
    global vocab, reverse_vocab, vocab_size
    tokens = text.lower().split()
    ids = []
    for t in tokens:
        if t not in vocab:
            vocab[t] = vocab_size
            reverse_vocab[vocab_size] = t
            vocab_size += 1
        ids.append(vocab[t])
    return torch.tensor(ids, dtype=torch.long, device=device)

def detokenize(ids):
    words = [reverse_vocab.get(i, "<unk>") for i in ids]
    return " ".join(words)

# ---------- EMBEDDINGS ----------
# Increased embedding size to account for potential vocabulary growth
word_embeddings = nn.Embedding(10000, d_model).to(device)
pos_embeddings = nn.Embedding(max_seq_len, d_model).to(device)

# ---------- TRANSFORMER COMPONENTS ----------
def attention(q, k, v, mask=None):
    scores = torch.matmul(q, k.transpose(-2, -1)) / (q.size(-1) ** 0.5)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    weights = F.softmax(scores, dim=-1)
    return torch.matmul(weights, v)

def multi_head_attention(x):
    B, T, C = x.shape
    head_dim = C // num_heads
    qkv = nn.Linear(C, C * 3, device=device)(x)
    q, k, v = qkv.chunk(3, dim=-1)

    # Reshape and transpose for multi-head attention
    q = q.view(B, T, num_heads, head_dim).transpose(1, 2)
    k = k.view(B, T, num_heads, head_dim).transpose(1, 2)
    v = v.view(B, T, num_heads, head_dim).transpose(1, 2)

    attn_out = attention(q, k, v)

    # Concatenate heads and apply output linear layer
    attn_out = attn_out.transpose(1, 2).contiguous().view(B, T, C)
    return nn.Linear(C, C, device=device)(attn_out)


def feed_forward(x):
    return nn.Sequential(
        nn.Linear(d_model, d_ff, device=device),
        nn.ReLU(),
        nn.Linear(d_ff, d_model, device=device)
    )(x)

# ---------- TRANSFORMER BLOCK ----------
def transformer_block(x):
    attn_out = multi_head_attention(x)
    x = x + attn_out
    x = nn.LayerNorm(d_model, device=device)(x)
    ff_out = feed_forward(x)
    x = x + ff_out
    x = nn.LayerNorm(d_model, device=device)(x)
    return x

# ---------- MINI LLM FORWARD ----------
def mini_llm_forward(input_ids):
    positions = torch.arange(0, input_ids.size(1), device=device) # Changed to use size(1) for batch processing
    x = word_embeddings(input_ids) + pos_embeddings(positions)
    x = transformer_block(x)
    logits = nn.Linear(d_model, vocab_size, device=device)(x) # Changed to use dynamic vocab_size
    return logits

# ---------- GENERATION ----------
def generate(prompt, max_new_tokens=15):
    ids = tokenize(prompt).unsqueeze(0) # Add batch dimension for forward pass
    for _ in range(max_new_tokens):
        # Ensure sequence length does not exceed max_seq_len
        if ids.size(1) >= max_seq_len:
            ids = ids[:, -max_seq_len:]
        logits = mini_llm_forward(ids)
        next_token_logits = logits[0, -1, :]
        probs = F.softmax(next_token_logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1).unsqueeze(0) # Add batch dimension
        ids = torch.cat([ids, next_id], dim=1) # Concatenate along the sequence dimension
        if next_id.item() == vocab.get("<eos>", -1):
            break
    return detokenize(ids.squeeze(0).tolist()) # Remove batch dimension for detokenization

# ---------- VECTOR STORE (SQLite) ----------
conn = sqlite3.connect("memory.db")
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS memory (text TEXT, vector BLOB)")
conn.commit()

def store_text(text):
    # Ensure text is not empty before processing
    if not text.strip():
        return
    ids = tokenize(text).unsqueeze(0) # Add batch dimension
    if ids.size(1) == 0: # Handle empty token list
        return
    with torch.no_grad(): # Use no_grad for inference
        vec = word_embeddings(ids).mean(dim=1).detach().cpu().numpy() # Mean along sequence dimension
    cur.execute("INSERT INTO memory VALUES (?, ?)", (text, vec.tobytes()))
    conn.commit()

def retrieve_similar(query, top_k=2):
    # Ensure query is not empty before processing
    if not query.strip():
        return []
    q_ids = tokenize(query).unsqueeze(0) # Add batch dimension
    if q_ids.size(1) == 0: # Handle empty token list
        return []
    with torch.no_grad(): # Use no_grad for inference
        q_vec = word_embeddings(q_ids).mean(dim=1).detach().cpu().numpy() # Mean along sequence dimension
    cur.execute("SELECT text, vector FROM memory")
    results = []
    for text, blob in cur.fetchall():
        vec = np.frombuffer(blob, dtype=np.float32).reshape(q_vec.shape) # Reshape loaded vector
        denom = (np.linalg.norm(q_vec) * np.linalg.norm(vec))
        sim = np.dot(q_vec.flatten(), vec.flatten()) / (denom + 1e-8) # Flatten for dot product
        results.append((sim, text))
    results.sort(reverse=True)
    return [t for _, t in results[:top_k]]

# ---------- DEMO ----------
store_text("The sun rises in the east.")
store_text("Artificial intelligence is the future.")
store_text("Cats are cute and love to sleep.")

query = "Tell me about the sun"
context = retrieve_similar(query)
prompt = " ".join(context) + " " + query
print("Prompt with context:\n", prompt)
print("\nGenerated:\n", generate(prompt))

Using device: cpu
Prompt with context:
 The sun rises in the east. Artificial intelligence is the future. Tell me about the sun

Generated:
 the sun rises in the east. artificial intelligence is the future. tell me about the sun and artificial future. in rises cats rises tell cats east. about are sun love and
